<a href="https://colab.research.google.com/github/CaioMendes92/Acompanhamento-Mensal-das-Transfer-ncias-Obrigat-rias-da-Uni-o-utilizando-BS4-Airflow/blob/main/Acompanhamento_Mensal_das_Transfer%C3%AAncias_Obrigat%C3%B3rias_da_Uni%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
from bs4 import BeautifulSoup
import urllib.request as urllib_request
from urllib.request import urlopen
import pandas as pd
import numpy  as np
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# funções

In [47]:
def webscrapping(url):
  response = urlopen(url)
  html = response.read()
  soup = BeautifulSoup(html, 'html.parser')
  links = []
  for item in soup.findAll('a', {"class": "resource-url-analytics"}):
    links.append(item.get('href'))
  df_final = pd.DataFrame()
  for i in range(2,18):
    df = pd.read_csv(links[i], encoding='ISO8859',skiprows=7, on_bad_lines='skip', sep=';')
    df = df.iloc[1:28]
    df['CSV Nome'] = links[i].split('/')[9].split('.')[0]
      
    df_final = pd.concat([df_final,df])
  return df_final

In [48]:
url = 'https://www.tesourotransparente.gov.br/ckan/dataset/transferencias-obrigatorias-da-uniao'
df = webscrapping(url)

In [49]:
df.to_csv('dados_brutos.csv')

In [50]:
def tratamento_dados(df_final,estado,ano_inicial,ano_final):
  df_final['ESTADOS'].fillna(df_final['Capitais'], inplace=True)
  df_final['ESTADOS'] = df_final['ESTADOS'].apply(lambda x: 
                                        'Acre' if x=="Rio Branco" else 
                                        'Alagoas' if x=='Maceió' else
                                        'Amazonas' if x=='Manaus' else
                                        'Amapá' if x=='Macapá' else
                                        'Bahia' if x=='Salvador' else
                                        'Ceará' if x=='Fortaleza' else
                                        'Distrito Federal' if x=='Brasília' else
                                        'Espírito Santo' if x=='Vitória' else
                                        'Goiás' if x=='Goiânia' else
                                        'Maranhão' if x=='São Luís' else
                                        'Minas Gerais' if x=='Belo Horizonte' else
                                        'Mato Grosso do Sul' if x=='Campo Grande' else
                                        'Mato Grosso' if x=='Cuiabá' else
                                        'Pará' if x=='Belém' else
                                        'Paraíba' if x=='João Pessoa' else
                                        'Pernambuco' if x=='Recife' else
                                        'Piauí' if x=='Teresina' else
                                        'Paraná' if x=='Curitiba' else
                                        'Rio Grande do Norte' if x=='Natal' else
                                        'Rondônia' if x=='Porto Velho' else
                                        'Roraima' if x=='Boa Vista' else
                                        'Rio Grande do Sul' if x=='Porto Alegre' else
                                        'Santa Catarina' if x=='Florianópolis' else
                                        'Sergipe' if x=='Aracaju' else 
                                        'Tocantins'if x=='Palmas' else x)
  df_final.drop('Capitais',axis=1, inplace=True)
  df_final = df_final.replace(' -   ', '0')
  dft = df_final[(df_final['ESTADOS'] == estado) ]\
  .melt(id_vars=['CSV Nome', 'ESTADOS','UF'], 
      value_vars = [col for col in df_final.columns 
                    if col not in ['CSV Nome', 'ESTADOS','UF']])
  mes_numbers = {'janeiro':1,'fevereiro':2,'março':3,'abril':4,'maio':5,
                'junho':6,'julho':7,'agosto':8,'setembro':9,'outubro':10,
                'novembro':11,'dezembro':12}

  init_years_range = ano_inicial
  final_years_range = ano_final

  df_all = ''

  df_final = df_final.applymap(lambda x: x.strip() if isinstance(x, str) else x)
  df_final = df_final.applymap(lambda x: x.replace('.','') if isinstance(x, str) else x)
  df_final = df_final.applymap(lambda x: x.replace(',','.') if isinstance(x, str) else x)

  for year in range(init_years_range,final_years_range+1):
    cols = ['CSV Nome', 'ESTADOS','UF'] + [col for col in df_final.columns if 
                                          col.endswith(f'/{year}')]
    dft  = df_final.loc[:,cols]

    dft = dft.melt(id_vars=['CSV Nome', 'ESTADOS','UF'], 
          value_vars = [col for col in dft.columns 
                        if col not in ['CSV Nome', 'ESTADOS','UF']])

    dft[['Mês','Ano']] = dft['variable'].str.split('/',expand=True)
    
    dft.drop(['variable','Ano'],inplace=True,axis=1)
    dft.rename(columns={'value':f'20{year}'},inplace=True)
    dft['Mês'] = dft['Mês'].replace(mes_numbers)
    dft = dft[['CSV Nome', 'ESTADOS','UF','Mês', f'20{year}']]
    
    if isinstance(df_all,str) == True:
      df_all = dft
    else:
      df_all = df_all.merge(dft,on=['CSV Nome','ESTADOS','Mês','UF'],how='left')
    df_all[f'20{year}'] = df_all[f'20{year}'].astype('float64')
  return df_all[df_all['ESTADOS'] == estado].fillna(0)

In [51]:
df2 = tratamento_dados(df,estado='Alagoas', ano_inicial=17, ano_final=22)
df2

,CSV Nome,ESTADOS,UF,Mês,2017,2018,2019,2020,2021,2022
1,CIDEEST,Alagoas,AL,1,4492196.43,4485114.67,2295681.62,2141474.89,966410.06,2230921.30
28,CIDEMUN,Alagoas,AL,1,1497398.81,1495038.22,765227.21,713824.96,322136.69,743640.43
55,FEXEST,Alagoas,AL,1,0.00,0.00,0.00,0.00,0.00,0.00
82,FEXMUN,Alagoas,AL,1,0.00,0.00,0.00,0.00,0.00,0.00
109,FPE,Alagoas,AL,1,242256259.13,256938067.84,316145617.39,284004528.95,343522250.88,425107461.45
...,...,...,...,...,...,...,...,...,...,...
5050,LC87EST,Alagoas,AL,12,819214.51,802410.11,0.00,0.00,0.00,0.00
5077,LC87MUN,Alagoas,AL,12,273071.89,267470.41,0.00,0.00,0.00,0.00
5104,LC176EST,Alagoas,AL,12,0.00,0.00,0.00,13501500.00,1374900.06,0.00
5131,LC176MUN,Alagoas,AL,12,0.00,0.00,0.00,2193476.20,452731.20,0.00


In [52]:
df2.dtypes

CSV Nome     object
ESTADOS      object
UF           object
Mês           int64
2017        float64
2018        float64
2019        float64
2020        float64
2021        float64
2022        float64
dtype: object

In [53]:
df2.to_csv('dados_tratados_alagoas.csv')